In [ ]:
# Homework 5 - AB Test

In [ ]:
!pip3 install google.analytics.data
!pip3 install pingouin

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.4/188.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.4/204.4 kB 6.0 MB/s eta 0:00:00


In [ ]:
from google.analytics.data_v1beta import BetaAnalyticsDataClient
from google.analytics.data_v1beta.types import (
    DateRange,
    Dimension,
    Metric,
    RunReportRequest,
    Filter,
    FilterExpression,
)
import os
import pandas as pd
import json
import scipy.stats as st
import numpy as np
from pingouin import ttest
from google.colab import userdata


# Project 1
Compare the session numbers for the two campaigns: one with medium being "announcement" and the other with medium being "canvas". Perform statistical tests to compare the difference.

Please make sure to upload the json file to the working folder.

## Problem 1.1

Construct the function sample_run_report:
- keep dimensions: source, medium, date
- keep metric: sessions
- date range: start_date="2025-01-01", end_date="today"
- dimension filter: "source"="social"

Expected Output:
- Code

In [ ]:
def sample_run_report(property_id="481940999"):#  424145747
    """Runs a simple report on a Google Analytics 4 property."""
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'bigquery-419120-14d56b3f70eb.json'
    client = BetaAnalyticsDataClient()
    request = RunReportRequest(
        property="properties/{}".format(property_id),
        dimensions=[Dimension(name="source"),Dimension(name="medium"),Dimension(name="campaignName"),Dimension(name="date")],
        metrics=[Metric(name="sessions")],
        date_ranges=[DateRange(start_date="2025-01-01", end_date="today")],
    )
    response = client.run_report(request)
    return response

## Problem 1.2 Convert the resulting data into a dataframe
Expected Output:
- Code

In [ ]:
def response_to_df(response):
    columns = []
    rows = []

    for col in response.dimension_headers:
        columns.append(col.name)
    for col in response.metric_headers:
        columns.append(col.name)

    for row_data in response.rows:
        row = []
        for val in row_data.dimension_values:
            row.append(val.value)
        for val in row_data.metric_values:
            row.append(val.value)
        rows.append(row)
    return pd.DataFrame(rows, columns=columns)



response=sample_run_report(property_id="481940999")
df=response_to_df(response)
print(df)

              source    medium campaignName      date sessions
0           (direct)    (none)     (direct)  20250326       32
1           (direct)    (none)     (direct)  20250323       17
2           (direct)    (none)     (direct)  20250325       10
3           (direct)    (none)     (direct)  20250327       10
4             wechat    social   Grow_Power  20250324        9
..               ...       ...          ...       ...      ...
72            wechat    social        hello  20250325        1
73            wechat    social        hello  20250327        1
74            wechat    social        hello  20250328        1
75  weixin110.qq.com  referral   (referral)  20250324        1
76  weixin110.qq.com  referral   (referral)  20250403        1

[77 rows x 5 columns]


## Problem 1.3 Prepare Data

In [ ]:
canvas = df[df['campaignName'] == 'Grow_Power']['sessions'].astype(int)

announcement =df[df['campaignName'] == 'hello']['sessions'].astype(int)

## Problem 1.4 Run Student's T-test and Welch's T-test.

In [ ]:
from pingouin import ttest
# Student's T-test
ttest(canvas,announcement,correction=False)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,3.109926,6,two-sided,0.020851,"[1.02, 8.58]",2.271169,3.205,0.736702


In [ ]:
# Welch's T-test
ttest(canvas,announcement,correction=True)

## Problem 1.5 Compare the tests and make conclusions.

```
+----------------------------+
| Input Your Answers Here.   |
 1. different assumptions
 2. different results
+----------------------------+
```

## Problem 1.6 Change a Metric and Perform T-tests Again
From this page (https://developers.google.com/analytics/devguides/reporting/data/v1/api-schema), choose a metric that is different from sessions. Explain why your chosen metric is of significance and run statistical tests to tell whether these two channels are different regarding your chosen metric.

In [ ]:

```
+----------------------------+
| Input Your Answers Here.   |
 1. a metric that is compatible with sessions
 2. justify your choice
+----------------------------+
```


# Project 2
In this project, you are asked to perform statistical tests to see whether two simulated samples have the same mean statistically.

## Problem 2.1
The two samples below have the same level of variance (standard deviation=5) and sample size but different means (25 and 30).

Based on the data, run both Student's t-test and Welch's t-test.
In your answer, are these two samples' means statistically different?
Do you get different results from Student's t-test and Welch's t-test?

In [ ]:
import numpy as np
from scipy import stats

# Simulated data for the control group (the mean is 25, the standard deviation is 5, and the sample size is 1000)
control_group = np.random.normal(loc=25, scale=5, size=1000) #

# Simulated data for the treatment group (the mean is 30, the standard deviation is 5, and the sample size is 1000)
treatment_group = np.random.normal(loc=30, scale=5, size=1000)

In [ ]:
#!pip3 install pingouin
from pingouin import ttest

ttest(treatment_group,control_group,correction=False)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,21.37462,1998,two-sided,1.868109e-91,"[4.28, 5.14]",0.955902,4.382e+87,1.0


In [ ]:
ttest(treatment_group,control_group,correction=True)

```
+----------------------------+
| Input Your Answers Here.   |
 1. Given the ground truth, we know which test is applicable
 2. Compare the results to justify your argument
+----------------------------+
```

## Problem 2.2

The two samples below have different sample sizes, means, and variances.

Based on the data, run both Student's t-test and Welch's t-test. Are these two samples' means statistically different?

Do you get different results from Student's t-test and Welch's t-test? Which test is more reliable? Why?

In [ ]:
control_group = np.random.normal(loc=25, scale=3, size=60)
treatment_group = np.random.normal(loc=30, scale=20, size=100)

In [ ]:
from pingouin import ttest

ttest(treatment_group,control_group,correction=True)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,3.418108,104.533025,two-sided,0.0009,"[3.06, 11.52]",0.435676,33.403,0.755455


In [ ]:
from pingouin import ttest

ttest(treatment_group,control_group,correction=False)

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,2.667962,158,two-sided,0.008427,"[1.89, 12.69]",0.435676,4.433,0.755455


```
+----------------------------+
| Input Your Answers Here.   |
 1. Given the ground truth, we know which test is applicable
 2. Compare the results to highlight the importance of assumptions and the choice of tests.
 3. Highlight which test is more reliable in this case.
+----------------------------+
```